# 🤖 OpenAI ChatGPT API와 LangChain 활용 완전 가이드

## 📚 개요

이 튜토리얼에서는 **OpenAI의 ChatGPT API**를 **LangChain 프레임워크**와 함께 활용하는 방법을 단계별로 학습합니다. 

### 🎯 학습 목표

- **LangChain**과 **LCEL(LangChain Expression Language)** 기본 개념 이해
- **ChatOpenAI** 클래스 활용법 완전 정복
- **다양한 모델 옵션**과 **설정 방법** 이해
- **멀티모달(이미지 인식)** 기능 활용
- **스트리밍** 및 **프롬프트 엔지니어링** 실습

### 📖 목차

1. **🛠️ 환경 설정** - 기본 설정 및 LangSmith 연동
2. **🚀 LangChain과 LCEL 기본 개념** - 핵심 개념 이해
3. **🤖 ChatOpenAI 기본 사용법** - 객체 생성과 기본 질의
4. **📊 응답 구조 이해** - AI 메시지 형태와 메타데이터
5. **🎯 고급 기능** - LogProb, 스트리밍, 멀티모달
6. **✨ 프롬프트 엔지니어링** - System/User 프롬프트 활용

### 💡 사전 준비사항

- OpenAI API 키 발급 및 환경변수 설정
- Python 가상환경 구성 완료
- 기본적인 Python 문법 이해

---

## 🛠️ 환경 설정

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# .env 파일에 LANGCHAIN_API_KEY를 입력합니다.
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

---

## 🚀 LangChain과 LCEL 기본 개념

### 🔗 LangChain이란?

**LangChain**은 Large Language Model(LLM)을 활용한 애플리케이션 개발을 위한 **통합 프레임워크**입니다. 

#### 🏗️ LangChain의 핵심 가치

- **🔧 표준화**: 다양한 LLM을 동일한 인터페이스로 사용
- **⚡ 효율성**: 복잡한 AI 워크플로우를 간단한 코드로 구현
- **🔄 재사용성**: 한 번 만든 컴포넌트를 다양한 곳에 활용
- **🛠️ 확장성**: 필요에 따라 기능을 쉽게 추가/변경

### ⚗️ LCEL(LangChain Expression Language)

**LCEL**은 LangChain 컴포넌트들을 **체인처럼 연결**하는 강력한 문법입니다.

#### 🎯 LCEL의 특징

```python
# 전통적인 방식
def traditional_approach(input_text):
    prompt_result = prompt.format(input=input_text)
    llm_result = llm.invoke(prompt_result)
    output_result = output_parser.parse(llm_result)
    return output_result

# LCEL 방식 - 간결하고 명확!
chain = prompt | llm | output_parser
result = chain.invoke({"input": input_text})
```

#### ✨ LCEL의 장점

- **📖 가독성**: 처리 흐름을 한눈에 파악 가능
- **⚡ 병렬 처리**: 자동으로 최적화된 실행
- **🔄 스트리밍**: 실시간 결과 출력 지원
- **🛡️ 오류 처리**: 강력한 예외 처리 메커니즘

### 💻 실제 활용 예시

```python
# 프롬프트 → LLM → 결과 파싱의 연쇄 작업
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 각 컴포넌트 정의
prompt = ChatPromptTemplate.from_template("Explain {topic} in simple terms")
llm = ChatOpenAI(model="gpt-4.1")
parser = StrOutputParser()

# LCEL로 체인 구성
chain = prompt | llm | parser

# 한 번의 호출로 전체 과정 실행
result = chain.invoke({"topic": "artificial intelligence"})
```

이제 실제 **ChatOpenAI** 클래스를 활용해보겠습니다! 🎯

---

## 🤖 ChatOpenAI - OpenAI 모델의 LangChain 인터페이스

**ChatOpenAI**는 OpenAI의 강력한 언어 모델들을 LangChain에서 쉽게 사용할 수 있도록 해주는 **통합 인터페이스**입니다.

### 🎯 주요 설정 옵션들

#### 🌡️ **temperature** (창의성 조절)
```python
temperature=0.1  # 일관되고 정확한 답변 (사실 질문에 적합)
temperature=0.8  # 창의적이고 다양한 답변 (창작 활동에 적합)
```
- **범위**: 0.0 ~ 2.0
- **낮은 값 (0.0~0.3)**: 정확하고 일관된 답변, 팩트 체크에 최적
- **중간 값 (0.4~0.7)**: 균형잡힌 답변, 일반적인 대화에 적합  
- **높은 값 (0.8~2.0)**: 창의적이고 예측 불가능한 답변, 브레인스토밍에 유용

#### 📏 **max_tokens** (최대 출력 길이)
```python
max_tokens=2048  # 최대 2048개 토큰까지 생성
```
- **토큰이란?** 단어의 일부분 또는 전체 (한국어 1토큰 ≈ 2-3글자)
- **비용 절약 팁**: 필요한 만큼만 설정하여 API 비용 최적화

#### 🏷️ **model_name** (사용할 모델)
- **GPT-4.1**: 최신 고성능 모델, 복잡한 추론과 멀티모달 지원
- **GPT-4.1-mini**: 성능과 비용의 균형, 일반적인 작업에 최적
- **GPT-4.1-nano**: 빠르고 저렴한 모델, 간단한 작업에 적합

### 📊 OpenAI 모델 비교표

| 모델 계열       | 모델명 (API Name) | 입력       | 컨텍스트 윈도우  | 최대 출력 토큰 | 지식 마감일 (Cutoff) | 가격 (1M토큰당)                  |
| :---------- | :------------- | :------- | :-------- | :------- | :-------------- | :------------------------------ |
| **GPT-5** ⭐   | `gpt-5`        | 텍스트, 이미지 | 400,000   | 128,000  | 2024년 9월 30일    | **입력:** $1.25<br>**출력:** $10.00 |
|             | `gpt-5-mini`   | 텍스트, 이미지 | 400,000   | 128,000  | 2024년 5월 31일    | **입력:** $0.25<br>**출력:** $2.00  |
|             | `gpt-5-nano`   | 텍스트, 이미지 | 400,000   | 128,000  | 2024년 5월 31일    | **입력:** $0.05<br>**출력:** $0.40  |
| **GPT-4.1** 🚀 | `gpt-4.1`      | 텍스트, 이미지 | 1,047,576 | 32,768   | 2024년 6월 1일     | **입력:** $2.00<br>**출력:** $8.00  |
|             | `gpt-4.1-mini` | 텍스트, 이미지 | 1,047,576 | 32,768   | 2024년 6월 1일     | **입력:** $0.40<br>**출력:** $1.60  |
|             | `gpt-4.1-nano` | 텍스트, 이미지 | 1,047,576 | 32,768   | 2024년 6월 1일     | **입력:** $0.10<br>**출력:** $0.40  |
| **GPT-4o**  📷  | `gpt-4o`       | 텍스트, 이미지 | 128,000   | 16,384   | 2023년 10월 1일    | **입력:** $2.50<br>**출력:** $10.00 |
|             | `gpt-4o-mini`  | 텍스트, 이미지 | 128,000   | 16,384   | 2023년 10월 1일    | **입력:** $0.15<br>**출력:** $0.60  |

![OpenAI Models Comparison](./images/gpt-models3-202508.png)

### 💡 모델 선택 가이드

- **🎯 정확성 우선**: `gpt-4.1` - 복잡한 분석, 전문적 작업
- **⚡ 균형잡힌 선택**: `gpt-4.1-mini` - 일반적인 용도, 가성비 최고
- **💰 비용 절약**: `gpt-4.1-nano` - 간단한 질답, 대량 처리

> **참고 링크**: [OpenAI 공식 모델 문서](https://platform.openai.com/docs/models)

### 🚀 기본 사용법

이제 실제로 ChatOpenAI를 사용해보겠습니다!

In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0) - 낮을수록 일관된 답변
    model_name="gpt-4.1",  # 사용할 OpenAI 모델명
)

# 사용자 질의내용 정의
question = "대한민국의 수도는 어디인가요?"

# LLM에 질의하고 결과 출력
print(f"[답변]: {llm.invoke(question)}")

### 📊 응답 구조 이해하기 - AIMessage 객체

ChatOpenAI의 응답은 단순한 텍스트가 아닌 **AIMessage 객체**로 반환됩니다. 이 객체에는 다양한 유용한 정보가 담겨있습니다!

#### 🔍 AIMessage의 구성 요소

- **💬 content**: 실제 AI가 생성한 텍스트 답변
- **📊 response_metadata**: 토큰 사용량, 모델 정보, 처리 시간 등의 메타데이터
- **🏷️ 기타 정보**: 메시지 ID, 타입 등

In [ ]:
# 사용자 질의내용 정의
question = "대한민국의 수도는 어디인가요?"

# LLM에 질의하고 응답 객체를 변수에 저장
response = llm.invoke(question)

In [ ]:
# 전체 응답 객체 확인 (AIMessage 형태)
response

In [ ]:
# 응답 내용만 텍스트로 추출
response.content

In [ ]:
# 응답 메타데이터 확인 (토큰 사용량, 모델 정보 등)
response.response_metadata

### 🎯 LogProb - AI의 확신도 측정하기

**LogProb(로그 확률)** 은 AI가 각 단어를 선택할 때의 **확신도**를 수치로 보여주는 고급 기능입니다.

#### 🤔 LogProb이 유용한 상황들

- **📊 답변 신뢰도 평가**: AI가 얼마나 확신하는지 판단
- **🔍 품질 관리**: 불확실한 답변을 필터링
- **📈 모델 분석**: AI의 의사결정 과정 이해
- **⚖️ 여러 답변 비교**: 가장 확실한 답변 선택

#### 💡 LogProb 해석 방법

- **높은 확률 (-0.1 ~ 0.0)**: AI가 매우 확신하는 답변
- **중간 확률 (-2.0 ~ -0.1)**: 일반적인 수준의 확신
- **낮은 확률 (-5.0 이하)**: AI가 불확실해하는 답변

> **💭 실생활 비유**: 시험에서 답을 고를 때 "90% 확신" vs "50% 확신"과 같은 개념입니다!

In [ ]:
# LogProb 기능이 활성화된 ChatOpenAI 객체 생성
llm_with_logprob = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 출력 토큰 수
    model_name="gpt-4.1",  # 사용할 OpenAI 모델명
).bind(logprobs=True)  # 토큰별 확률 정보 활성화

In [ ]:
# 사용자 질의내용 정의
question = "대한민국의 수도는 어디인가요?"

# LogProb가 활성화된 LLM으로 질의
response = llm_with_logprob.invoke(question)

In [ ]:
# LogProb 정보가 포함된 메타데이터 확인
response.response_metadata

### ⚡ 스트리밍 출력 - 실시간 응답 체험

**스트리밍**은 AI가 답변을 **실시간으로 생성하면서 즉시 보여주는** 기능입니다. 마치 사람이 말하듯이 단어 하나씩 나타나는 것을 볼 수 있습니다!

#### 🎯 스트리밍이 유용한 이유

- **⏰ 즉시성**: 긴 답변도 기다림 없이 바로 확인
- **🔍 투명성**: AI가 어떻게 생각하는지 과정을 관찰
- **💬 자연스러움**: 실제 대화하는 듯한 경험
- **⚡ 효율성**: 전체 답변 완료를 기다릴 필요 없음

#### 💡 활용 시나리오

- **📝 긴 글 작성**: 에세이, 보고서 등
- **💬 실시간 챗봇**: 대화형 애플리케이션
- **🎥 라이브 데모**: 시연이나 발표 시
- **⏱️ 긴급 질문**: 빠른 피드백이 필요할 때

In [ ]:
# 스트리밍 방식으로 LLM에 질의
# 실시간으로 토큰이 생성되는 과정을 확인할 수 있음
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")

In [ ]:
# 스트리밍 응답을 실시간으로 출력
# 각 토큰이 생성될 때마다 즉시 화면에 표시됨
for token in answer:
    print(token.content, end="", flush=True)

In [ ]:
from langchain_teddynote.messages import stream_response

# 스트리밍 방식으로 LLM에 질의
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")

# langchain_teddynote의 stream_response 함수로 깔끔하게 출력
stream_response(answer)

## 📷 멀티모달 AI - 이미지를 읽는 인공지능

**멀티모달(Multimodal)** 은 여러 종류의 데이터를 동시에 처리할 수 있는 AI 기술입니다. **텍스트뿐만 아니라 이미지, 오디오, 비디오**까지 이해할 수 있습니다!

### 🌈 멀티모달이 처리할 수 있는 데이터 타입

- **📝 텍스트**: 문서, 이메일, 웹페이지 등의 글자 정보
- **🖼️ 이미지**: 사진, 그래프, 도표, 스크린샷 등의 시각적 정보  
- **🎵 오디오**: 음성, 음악, 효과음 등의 청각적 정보
- **🎬 비디오**: 동영상, 애니메이션 등 시청각 복합 정보

### 🎯 GPT-4.1의 비전(Vision) 기능

**GPT-4.1**은 강력한 **이미지 인식 능력**을 갖춘 멀티모달 모델입니다.

#### 🔍 이미지 분석 가능한 작업들

- **📊 차트/그래프 해석**: 데이터 시각화 분석
- **📄 문서 OCR**: 이미지 속 텍스트 추출 및 해석
- **🏞️ 장면 설명**: 사진 속 상황과 객체 인식
- **📋 표/양식 처리**: 복잡한 테이블 데이터 이해
- **🎨 창작물 분석**: 그림, 디자인 요소 해석

#### 💼 실제 비즈니스 활용 사례

- **📈 재무제표 분석**: 복잡한 회계 자료 자동 해석
- **🏥 의료 영상 검토**: X-ray, MRI 이미지 보조 분석
- **🏗️ 건축 도면 검토**: 설계도 및 시공 현황 파악
- **📦 제품 관리**: 상품 사진을 통한 품질 검사

> **🚀 혁신 포인트**: 이제 "이 이미지에서 뭐가 보이나요?"가 아니라 "이 재무제표의 핵심 인사이트는 무엇인가요?"까지 질문할 수 있습니다!

In [ ]:
from langchain_teddynote.models import MultiModal
from langchain_teddynote.messages import stream_response

# 기본 ChatOpenAI 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4.1",  # 이미지 인식이 가능한 모델
)

# 멀티모달(이미지 + 텍스트 처리) 객체 생성
multimodal_llm = MultiModal(llm)

In [ ]:
# 웹상의 이미지 URL 정의
IMAGE_URL = "https://t3.ftcdn.net/jpg/03/77/33/96/360_F_377339633_Rtv9I77sSmSNcev8bEcnVxTHrXB4nRJ5.jpg"

# 웹 이미지를 직접 분석하여 스트리밍 응답 생성
answer = multimodal_llm.stream(IMAGE_URL)

# 실시간으로 이미지 분석 결과 출력
stream_response(answer)

In [ ]:
# 로컬 저장된 이미지 파일 경로 정의
IMAGE_PATH_FROM_FILE = "./images/sample-image.png"

# 로컬 이미지 파일을 분석하여 스트리밍 응답 생성
answer = multimodal_llm.stream(IMAGE_PATH_FROM_FILE)

# 실시간으로 이미지 분석 결과 출력
stream_response(answer)

## ✨ 프롬프트 엔지니어링 - AI의 역할과 행동 설계

**프롬프트 엔지니어링**은 AI가 원하는 방식으로 동작하도록 **정확한 지시사항을 설계**하는 기술입니다. 마치 전문가를 고용할 때 **업무 설명서**를 작성하는 것과 같습니다!

### 🎭 System Prompt vs User Prompt

#### 👨‍💼 **System Prompt** - AI의 정체성과 역할 정의
```python
system_prompt = "You are a professional financial analyst..."
```
- **목적**: AI가 **누구인지, 어떤 전문성을 가져야 하는지** 정의
- **특징**: 전체 대화 세션 동안 지속되는 **기본 설정**
- **예시**: "당신은 친절한 고객 서비스 직원입니다", "당신은 전문 의료진입니다"

#### 🗣️ **User Prompt** - 구체적인 작업 지시사항
```python
user_prompt = "Please analyze the financial data and provide insights..."
```
- **목적**: **구체적으로 무엇을 해야 하는지** 상세한 작업 명령
- **특징**: 각 요청마다 달라질 수 있는 **가변적 지시사항**
- **예시**: "이 문서를 요약해주세요", "3가지 대안을 제시해주세요"

### 🎯 효과적인 프롬프트 작성 원칙

#### 1️⃣ **명확성 (Clarity)**
```python
# ❌ 애매한 지시
"재무제표를 봐주세요"

# ✅ 명확한 지시  
"재무제표의 수익성 지표를 분석하고 3가지 핵심 인사이트를 제시해주세요"
```

#### 2️⃣ **구체성 (Specificity)**
```python
# ❌ 추상적 요청
"도움을 주세요"

# ✅ 구체적 요청
"매출 증감 원인을 분석하고, 향후 3개월 예측과 개선 방안을 제시해주세요"
```

#### 3️⃣ **맥락 제공 (Context)**
```python
# ✅ 풍부한 맥락
"당신은 10년 경력의 재무 분석 전문가입니다. 중소기업 CEO를 대상으로 복잡한 재무 용어는 쉽게 설명해주세요."
```

### 💡 프롬프트 엔지니어링의 실제 효과

좋은 프롬프트는 **AI의 성능을 10배 이상 향상**시킬 수 있습니다:

- **🎯 정확도 향상**: 원하는 답변을 더 정확하게 생성
- **📊 일관성 보장**: 매번 비슷한 품질의 결과 제공  
- **⚡ 효율성 증대**: 반복 질문과 수정 작업 최소화
- **🔧 커스터마이징**: 특정 업무나 스타일에 맞춤 조정

이제 실제 재무제표 분석에 특화된 프롬프트를 만들어보겠습니다! 🚀

In [ ]:
# 시스템 프롬프트: AI의 역할과 행동 방식을 정의
system_prompt = """You are a professional financial AI assistant specialized in analyzing financial statements and tables. 
Your mission is to interpret given tabular financial data and provide insightful, interesting findings in a friendly and helpful manner. 
Focus on key metrics, trends, and notable patterns that would be valuable for business analysis."""

# 사용자 프롬프트: 구체적인 작업 지시사항
user_prompt = """Please analyze the financial statement provided in the image. 
Identify and summarize the most interesting and important findings, including key financial metrics, trends, and insights that would be valuable for business decision-making."""

# 커스텀 프롬프트가 적용된 멀티모달 객체 생성
multimodal_llm_with_prompt = MultiModal(
    llm, 
    system_prompt=system_prompt,  # 시스템 역할 정의
    user_prompt=user_prompt       # 사용자 요청 정의
)

In [ ]:
# 분석할 재무제표 이미지 URL
IMAGE_PATH_FROM_FILE = "https://storage.googleapis.com/static.fastcampus.co.kr/prod/uploads/202212/080345-661/kwon-01.png"

# 커스텀 프롬프트가 적용된 멀티모달 LLM으로 재무제표 분석
answer = multimodal_llm_with_prompt.stream(IMAGE_PATH_FROM_FILE)

# 재무제표 분석 결과를 실시간으로 출력
stream_response(answer)